# Projetando um PID

## Modelando o sistema

Este trabalho foi baseado no problema 3.17 do livro *Control Systems* de *Norman S. Nise*, Sétima edição.

![Modelando o vôo do míssil](img/missil.png)

O problema modela o vôo de um míssil, que está sujeito a quatro forças: empuxo (*thrust*), sustentação (*lift*),
arrasto (*drag*) e gravidade. O míssil voa com um ângulo de ataque, $\alpha$, do seu eixo longitudinal, criando sustentação. Para seguir um determinado rumo, o ângulo do corpo da vertical, $\phi$, é controlado rotacionando o motor na cauda. 

A função de transferência relaciona o ângulo do corpo, $\phi$, e sua posição angular, $\delta$, do motor na forma, como mostrado na equação abaixo:

\begin{align}
 \frac{\Phi(s)}{\delta(s)} = \frac{K_a s + K_b}{K_3 s^3 + K_2 s^2 + K_1 s + K_0} \label{eq:plant}
\end{align}

## Entendendo a planta

Desejamos projetar um controlador PID para controlar o míssil, conforme relacionado pelo diagrama de blocos abaixo:

![Diagrama de blocos](img/block-diagram.svg)

Para fins práticos, escolheremos os parâmetros da planta de forma arbitrária.

A resposta ao degrau e impulso da planta são mostrados abaixo:

In [27]:
%matplotlib ipympl
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
from control import (TransferFunction, step_response, bode_plot,
                     impulse_response, series, feedback, rlocus,
                     margin, nyquist_plot)

ka, kb,  = [1, 5]
k3, k2, k1, k0 = [2, 50, 10, 10]

plant_tf = TransferFunction([ka, kb], [k3, k2, k1, k0])

In [28]:
def plot_step_response(tf, title='Step response', show_points=False):
    time, output = step_response(tf)
    fig = plt.figure()
    plt.plot(time, output.flatten())
    if show_points:
        plt.plot(time, output,'om')
    plt.title(title)
    plt.show()
    return time,output.flatten()

time,output = plot_step_response(plant_tf, 
                                 title='Plant step response')

FigureCanvasNbAgg()

E também podemos visualizar a resposta em frequência da planta através do diagrama de *Bode*:

In [29]:
plt.figure()
plt.title("Bode plot")
mag, phase, omega = bode_plot(plant_tf)

FigureCanvasNbAgg()

## Escolhendo parâmetros para o controlador

### Método resposta a frequência de Ziegler-Nichols
A fim de aplicarmos este método, acharemos o ganho crítico do sistema. Para isso analisaremos o lugar das raízes:

In [30]:
loci = rlocus(plant_tf, Plot=True, PrintGain=True)

FigureCanvasNbAgg()

Porém, percebe-se que nosso sistema não tem um ganho crítico, podemos aumentá-lo indefinidamente. Logo, não podemos utilizar este método.

### Método resposta ao degrau de Ziegler-Nichols

Para este método, precisamos achar o ponto de inflexão da curva para podermos calcular dois parâmetros: $L$ e $\alpha$. Destes dois 
parâmetros, projetaremos um P, PI ou PID seguindo as regras da tabela:

\begin{array}{rr} \hline
\text{Controlador} & K &T_i&T_d\\ \hline
\text{P} &1/\alpha&&& \\ \hline
\text{PI} &0.9/\alpha&3L&& \\ \hline
\text{PID} &1.2/\alpha&2L&L/2& \\ \hline
\end{array}

E a seguinte função transferência:
\begin{align}
\frac{\Delta(s)}{E(s)} = K_c \left( 1 + \frac{1}{T_i s} + T_d s \right)
\end{align}

Para encontrar estes parâmetros, revisitaremos a resposta ao degrau 
e escolheremos o ponto de inflexão para traçar a tangente.

In [31]:
time, output = plot_step_response(plant_tf, 
                                  title='Plant step response',
                                  show_points=True)

FigureCanvasNbAgg()

Escolhemos o quinto ponto como ponto de flexão, e vamos tracejar a linha tangente a ele, através da sua derivada (discreta).

In [32]:
def derivate_around(x,y,index):
    return (y[index] - y[index - 1])/(x[index]- x[index - 1])

def tangent_line(x, y, index):
    return y[index]+derivate_around(x, y, index)*(x - x[index])

inflection_index = 5
plt.figure()
plt.title("Inflection point tangent line")
axes = plt.gca()
axes.set_xlim([0,25])
axes.set_ylim([-0.5,2])
plt.grid(True)
plt.plot(time,output,'b',time,tangent_line(time, output, inflection_index),'--r')
plt.show()

FigureCanvasNbAgg()

Da figura acima obtemos $L$ e $\alpha$, da intersecção da reta tangente e o eixos $x$ e $y$, respectivamente:

\begin{align*}
L=0.86\\
\alpha=0.13
\end{align*}

In [33]:
def ziegler_nichols_constants(l,alpha):
    return (1.2/alpha, 2*l, l/2)

l = 0.86
alpha = 0.13
k, ti, td = ziegler_nichols_constants(l, alpha) 

(k, ti, td)

(9.23076923076923, 1.72, 0.43)

Logo, temos:

\begin{align}
K_c &= 9.23 \\
T_i &= 1.72 \\
T_d &= 0.43 \\
\frac{\Delta(s)}{E(s)} &= 9.23 \left( 1 + \frac{1}{1.72 s} + 0.43 s \right) = \frac{6.83 s^2 + 15.88s +9.23}{1.72 s}
\end{align}

In [34]:
controller_tf = TransferFunction([k*ti*td, k*ti, k],[ti, 0])
g = series(controller_tf, plant_tf)
system = feedback(g, 1)
time, output = plot_step_response(system)

FigureCanvasNbAgg()

In [35]:
plt.figure()
plt.title('Bode plot')
_ = bode_plot(g, margins=True)

FigureCanvasNbAgg()

/usr/lib/python3.7/site-packages/matplotlib/backends/backend_agg.py:211: RuntimeWarning: Glyph 8 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/usr/lib/python3.7/site-packages/matplotlib/backends/backend_agg.py:180: RuntimeWarning: Glyph 8 missing from current font.
  font.set_text(s, 0, flags=flags)


E então, olharemos métricas em frequência:

In [36]:
gm, pm, wg, wp = margin(g)
(gm, pm, wg, wp)

(inf, 16.968921242249138, nan, 1.0644410587221755)

Como o pacote "control" não implementa métricas no tempo, teremos que implementá-las. 

In [37]:
def settling_time(system, error=0.005):
    time, output = step_response(system)
    settling_time = None
    for t, out in zip(time, output.flatten()):
        if abs(1-out) < error:
            if settling_time is None:
                settling_time = t
        else:
                settling_time = None
    return settling_time

def rise_time(system, start=0, stop=1, precision=0.05):
    time, output = step_response(system)
    new_time, new_output = interpolate_resp(time, output.flatten(), start, stop)
    start_index = np.where(new_output >= start-precision)[0][0]
    stop_index = np.where(new_output >= stop-precision)[0][0]
    duration = new_time[stop_index]- new_time[start_index]
    return np.asscalar(duration)

def interpolate_resp(time, output, start, stop):
    interpolated = interp1d(time, output.flatten())
    inter_time = np.arange(0, 2*stop, stop/100)
    inter_output = np.asarray([interpolated(t) for t in inter_time])
    return inter_time, inter_output

def overshoot(system):
    time, output = step_response(system)
    overshoot = max(output.flatten())
    t_d = time[np.where( overshoot == output.flatten())]
    return np.asscalar(t_d), overshoot

def time_domain_metrics(system):
    _, sys_overshoot = overshoot(system)
    sys_settling = settling_time(system)
    sys_tr = rise_time(system)
    return (sys_overshoot, sys_settling, sys_tr)

time_domain_metrics(system)

(1.4669213618614014, 46.64508784878061, 1.32)

### Desempenho ziegler-nichols sem ajustes ($C_{pid}$)
\begin{align}
G_m &= \infty \\
P_m &= 16.97^{\circ} \\
M_p & = 1.47 \\
T_s & = 24.65 s \\
T_r & = 1.41s \\
\omega_p &= 1.06 rad/s
\end{align}

## Ajustes manuais nos parâmetros ($C_{pid_2}$)

A fim de diminuir o *overshoot* para abaixo de 20%, diminuiremos $T_i$ para um terço de seu valor
e multiplicaremos $T_d$ por 9.

In [38]:
ti_2 = ti/3
td_2 = td*9
controller2_tf = TransferFunction([k*ti_2*td_2, k*ti_2, k],[ti_2, 0])
g2 = series(controller2_tf, plant_tf)
system2 = feedback(g2, 1)
_, _ = plot_step_response(system2)

FigureCanvasNbAgg()

In [39]:
plt.figure()
plt.title('Bode plot')
_ = bode_plot(g2, margins=True)

FigureCanvasNbAgg()

In [40]:
sys2_overshoot, sys2_settling, sys2_tr = time_domain_metrics(system2)
gm2, pm2, wg2, wp2 = margin(g2)

(gm2, pm2, sys2_overshoot,sys2_settling, sys2_tr, wp2)

(inf,
 122.42676661695816,
 1.0750719937618094,
 32.095442750715314,
 0.98,
 4.893614824832795)

### Desempenho ziegler-nichols com ajustes ($C_{pid_2}$)
\begin{align}
G_m &= \infty \\
P_m &= 122.43^{\circ} \\
M_p & = 1.075 \\
T_s & = 4.47 s \\
T_r & = 1.28 s \\
\omega_p &= 4.89 rad/s
\end{align}

## Design por compensadores

Gostaríamos de projetar um controlador usando compensadores de atraso e de avanço. Como restrição, gostaríamos de uma frequência de corte muito maior que obtida em $C_{pid_1}$. 

Vamos fazer o design de um compensador de atraso, que é da forma:
\begin{align}
C_{lag} = k_0 \frac{1+ sT_i}{sT_i}
\end{align}

Queremos uma frequência de corte, $\omega_p$, uma ordem de grandeza maior. Da nossa intuição de design anteriores, escolheremos $T_i$ uma década antes da frequência de corte desejada, ou seja:

\begin{align}
\frac{1}{T_i} = 1 \therefore T_i &= 10
\end{align}

E ajustaremos o ganho de acordo.

In [41]:
kc, ti = 350, 10
g_lead = kc*TransferFunction([ti, 1], [ti, 0])
g3 = series(g_lead, plant_tf)
system3 = feedback(g3, 1)
_, _ = plot_step_response(system3)

sys3_overshoot, sys3_settling, sys3_tr =time_domain_metrics(system3)
gm3, pm3, wg3, wp3 = margin(system3)

(gm3, pm3, sys3_overshoot, sys3_settling, sys3_tr, wp3)

FigureCanvasNbAgg()

(inf,
 84.23952310638708,
 1.0041702994520167,
 0.7109861804415457,
 0.68,
 10.643514260064746)

### Desempenho design por compensador de Lag ($C_{pid_3}$)
\begin{align}
G_m &= \infty \\
P_m &= 84.24^{\circ} \\
M_p & = 1.0 \\
T_s & = 0.71 s \\
T_r & = 1.28 s \\
\omega_p &= 10.64 rad/s
\end{align}

Como a margem de fase já é suficientemente grande, não há necessidade de projetarmos um compensador de avanço.

## Comparativo entre controladores

\begin{array}{rrrr} \hline
 & C_{pid_1} &C_{pid_2}&C_{pid_3}\\ \hline
G_m &\infty &\infty&\infty \\ \hline
P_m  &16.97^{\circ}&122.43^{\circ}&84.24^{\circ} \\ \hline
M_p &1.47 &1.075&1 \\ \hline
T_s &24.65 s&4.47 s&0.71 s \\ \hline
T_r &1.41s & 1.28 s&1.28 s\\ \hline
\omega_p&1.06 rad/s&4.89 rad/s&10.64  rad/s\\ \hline
\end{array}

## Projeto digital
Escolheremos nosso controlador $C_{pid3}$, pois ele obteve um melhor compromisso entre métricas. 

Sua função transferência é: 

In [42]:
g_lead


3500 s + 350
------------
    10 s

Que ao amostrada torna-se:

In [43]:
g_lead_d = g_lead.sample(0.01, method='bilinear')
g_lead_d


350.2 z - 349.8
---------------
     z - 1

dt = 0.01

Obteremos a equação a diferenças deste controlador, em que $\delta$ é a saída e entrada é $e$:
\begin{align} 
\frac{\Delta}{E} &= \frac{350.2z - 349.8}{z - 1} = \frac{350.2z - 349.8}{z - 1} \times \frac{z^{-1}}{z^{-1}} \\
\frac{\Delta}{E} &= \frac{350.2 - 349.8 z^-1}{1 - z^-1} \\
\Delta (1 - z^{-1}) &= E (350.2 - 349.8 z^-1) \\
\delta[t] &= \delta[t-1] + 350.2 e[t] - 349.8 e[t-1]
\end{align}

O resultado ainda é um sistema estável, como veremos abaixo:

In [44]:
plant_tf_d = plant_tf.sample(0.01, method='zoh')
g4 = series(g_lead_d, plant_tf_d)
system4 = feedback(g4, 1)
plot_step_response(system4, title='Discrete step response')

FigureCanvasNbAgg()

(array([0, 1, 2, 3, 4, 5, 6, 7, 8]),
 array([0.        , 0.95134646, 0.99427395, 0.99588404, 0.99630344,
        0.9966542 , 0.99697096, 0.9972577 , 0.9975173 ]))

In [52]:
gm4, pm4, wg4, wp4 = margin(system4)
sys4_overshoot, sys4_settling, sys4_tr = time_domain_metrics(system4)
(gm4, pm4, sys4_overshoot, sys4_settling, sys4_tr, wg4)

(0.9396486474755695, inf, 0.9975172967533581, 3, 1.0, 1.590689321125211)

### Desempenho controlador digital ($C_{pid_D}$)
\begin{align}
G_m &= 0.94 \\
P_m &= \infty \\
M_p & = 1.0 \\
T_s & = 3 s \\
T_r & = 1s \\
\omega_p &= 1.59 rad/s
\end{align}

### Comparativo dos controladores

\begin{array}{rrrrr} \hline
 & C_{pid_1} &C_{pid_2}&C_{pid_3} & C_{pid_D}\\ \hline
G_m &\infty &\infty&\infty & 0.94 \\ \hline
    P_m  &16.97^{\circ}&122.43^{\circ}&84.24^{\circ}& \infty \\ \hline
M_p &1.47 &1.075&1& 1 \\ \hline
T_s &24.65 s&4.47 s&0.71 s & 3 \\ \hline
T_r &1.41s & 1.28 s&1.28 s & 1\\ \hline
\omega_p&1.06 rad/s&4.89 rad/s&10.64  rad/s & 1.59 rad/s\\ \hline
\end{array}